In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 11.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from openai.types.beta.threads import ImageFileContentBlock

In [4]:
import os
API_KEY = os.environ["OPENAI_API_KEY"]

KeyError: 'OPENAI_API_KEY'

In [ ]:
from openai import OpenAI


client = OpenAI(api_key=)

assistant = client.beta.assistants.create(
  instructions="You are an AI assistant that will allow users to talk to their data-rich models. In the responses where you're running code, do not provide any additional response besides the code and the output of the code. I will provide you with json files that contain data that were either made from Speckle or from IfcOpenShell and you need to be able to read this data and perform analysis on it.",
  model="gpt-4o-2024-05-13",
  tools=[{"type": "code_interpreter"}]
)

In [ ]:
file = client.files.create(
  file=open("rac_basic_sample_project.json", "rb"),
  purpose='assistants'
)

thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Here's some data in json format. It contains some structural data.",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

from typing_extensions import override
from openai import AssistantEventHandler

# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

# Then, we use the `stream` SDK helper
# with the `EventHandler` class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Provide me with some sort of cool visualisation for the various types of elements that are present in the building.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()